# Introduction to Task Group Framework

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
from src import Task, TaskGroup, Flag
from src.pre_processing.interfaces.executor_interface import ExecutorInterface

import warnings
warnings.filterwarnings("ignore")

In [2]:
matrix = pd.DataFrame(np.identity(5, dtype=int), columns=[0, 1, 2, 3, 4])

def plot(df):
    print(df.to_string(index=False, header=False))
    
plot(matrix)

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [3]:
result = 3 * matrix + 1
plot(result)

4 1 1 1 1
1 4 1 1 1
1 1 4 1 1
1 1 1 4 1
1 1 1 1 4


In [4]:
class Slope(ExecutorInterface):
    def execute(self, dataToProcess: pd.DataFrame) -> pd.DataFrame:
        return  3 * dataToProcess

class Constant(ExecutorInterface):
    def __init__(self, constant: int):
        self.constant = constant
        
    def execute(self, dataToProcess: pd.DataFrame) -> pd.DataFrame:
        return self.constant + dataToProcess
    
result = Slope().execute(matrix)
result = Constant(1).execute(result)

plot(result)

4 1 1 1 1
1 4 1 1 1
1 1 4 1 1
1 1 1 4 1
1 1 1 1 4


In [5]:
task_slope = Task(name = "Slope", executor=Slope())
task_constant = Task(name = "Constant", executor=Constant(1))

result = task_slope.execute(matrix)
result = task_constant.execute(result)

plot(result)

4 1 1 1 1
1 4 1 1 1
1 1 4 1 1
1 1 1 4 1
1 1 1 1 4


In [6]:
task_slope = Task(name = "Slope", executor=Slope(), priority=1)
task_constant = Task(name = "Constant", executor=Constant(1), priority=2)

task_group = TaskGroup("Linear Function")
task_group.addAll([task_constant, task_slope])

task_group.print(priority=True)

Linear Function
	1 Slope
	2 Constant


In [7]:
result = task_group.execute(matrix)
plot(result)

4 1 1 1 1
1 4 1 1 1
1 1 4 1 1
1 1 1 4 1
1 1 1 1 4


In [8]:
flag_slope = Flag(name = "Flag_Slope")
flag_constant = Flag(name = "Flag_Constant")

task_slope = Task(name = "Slope", executor=Slope(), priority=1, flags=flag_slope)
task_constant = Task(name = "Constant", executor=Constant(1), priority=2, flags=flag_constant)

task_group = TaskGroup(name = "Linear Function")
task_group.addAll([task_constant, task_slope])

task_group.print(priority=True, flags=True)

Linear Function
	1 Slope: Flag_Slope
	2 Constant: Flag_Constant


In [9]:
result = task_group.process(matrix, flag_slope)
plot(result)

3 0 0 0 0
0 3 0 0 0
0 0 3 0 0
0 0 0 3 0
0 0 0 0 3


In [10]:
result = task_group.process(matrix, flag_constant)
plot(result)

2 1 1 1 1
1 2 1 1 1
1 1 2 1 1
1 1 1 2 1
1 1 1 1 2


In [11]:
flag_linear_function = Flag(name="Flag_Linear_Function", parents=[flag_slope, flag_constant])

result = task_group.process(matrix, flag_linear_function)
plot(result)

4 1 1 1 1
1 4 1 1 1
1 1 4 1 1
1 1 1 4 1
1 1 1 1 4
